In [1]:
# Print out the orders


In [6]:
import os
from pathlib import Path

# This is the file that was downloaded from https://newmarketchurch.breezechms.com/forms/entries/557986
weekly_orders = r"C:\Users\ralph\Downloads\PANTRY-ORDER-FORM-101923 (5).xlsx"
title_date = '10/19/2023'

last_run = datetime(2023, 10, 17, 15, 0)
runmode = 'all'
# runmode = 'update'

# Get the exported filename for use in the report.
filename = Path(weekly_orders)
name = filename.name.replace(filename.suffix, '', -1)

filename = 'NCC Food Pantry Orders' 
title = filename + ' ' + title_date

# This file is created by this script and is to be imported at the same web page.
upload_file = os.path.join(os.path.dirname(weekly_orders), filename +'.html')
# upload_file

In [7]:
import numpy as np
import pandas as pd
from datetime import datetime

# Read in the downloaded Excel file 
allorders = pd.read_excel(weekly_orders)
received = len(allorders.index)
print('{count} orders in input.'.format(count = received))

69 orders in input.


In [8]:
from datetime import datetime, timedelta

modes = {
    'all': 'All orders received this week',
    'update': 'All orders received since {date}'.format(date = last_run),
    'special': 'Scpecial criteria were used.'
    }

# Only process recent orders.
if runmode == 'all':
    allorders = allorders[pd.to_datetime(allorders['Date']) > datetime.strptime(title_date, '%m/%d/%Y') - timedelta(days=5)]
elif runmode == 'update':
    allorders = allorders[pd.to_datetime(allorders['Date']) > last_run]
else:
    allorders = allorders[pd.to_datetime(allorders['Date']) > datetime.today() - timedelta(days=7)]  

printed = len(allorders.index)
print('{count} orders filtered by date and time.'.format(count = printed))

32 orders filtered by date and time.


In [9]:
# Coalesce columns to simplify the order form.

# Combine first and last name.
allorders.insert(1, 'Name', allorders['First Name'] + ' ' + allorders['Last Name'])

# Format the Zip code correctly.
allorders['Zip'] = allorders['Zip'].astype(str).str.pad(5,fillchar='0')

# Combine address elements into a single address field.
allorders.insert(5, 'Address', allorders['Street Address'] + '<br>' + allorders['City'] +' '+ allorders['State']+' '+ allorders['Zip'])

# Drop the fields we combined.
allorders = allorders.drop(columns=['Date', 'First Name', 'Last Name', 'Street Address', 'City', 'State', 'Zip'])

# Replace "NaN" values with blanks.
allorders = allorders.replace(np.nan, '')
# allorders.columns

In [19]:
def formatshoppers(date, data):
    output = ''
    
    rowvars = {
        'key' : None,
        'val' : None
    }
    for _, row in data.iterrows():
        rowtext = ''
        # Format each field as a row in a table.
        for i in range(len(data.columns)):
            rowvars['key'] = data.columns[i]
            rowvars['val'] = row.iloc[i]
            rowtext += rowhtml.render(rowvars)
        
        # Create a page using the above table.
        output += shopperhtml.render({'date': date, 'data': rowtext, 'shoppername': row['Name']})
    
    return output

In [17]:
from jinja2 import Template

# Define HTML templates using Jinja2 for printing the data.

# Template for the whole report, including style.

# TODO: I don't think the pagination is working correctly. 
#       Make sure each new shopper starts a new page - never on the back of the prior page.
# TODO: Do we want the fonts and spacing to match the current sheets?
# TODO: Add name to second page.
orders = Template('''
<!DOCTYPE html>
<style>
    @media print {
        h1 { 
            break-before: recto; 
        } 
    }
    table {
        width: 100%;
    }
    td {
        border-bottom: 1px solid #ddd;
    }
</style>
<html>
<head>
<meta charset="ISO-8859-1">
<title>{{title}}</title>
</head>
<body>
{{body}}
</body>
</html>
''')

# Template for each shopper
# TODO: Should we make the title more useful? 
#       Different titles for main page and summary page?
shopperhtml = Template('''
<h1>PANTRY ORDER FORM for {{shoppername}} ({{date}})</h1>
<table>
    <colgroup>
        <col span="1" style="width: 35%;">
        <col span="1" style="width: 65%;">
    </colgroup>
{{data}}
</table>
''')

coversheet = Template('''
<h2>NCC Pantry Order Print Cover Sheet for {{date}}</h2>
<p>Order forms received this week: {{received}}</p>
<p>Unique Orders Printed: {{printed}}</p>
''')

sectionheader = '''
<h1>One Page Summaries for Refrigerated Items</h1>
'''

# Template for each row
rowhtml = Template('<tr><td>{{key}}</td><td>{{val}}</td></tr>')


In [13]:
# Collect the summary for only refrigerated items. ("page 1")
# Include only the following fields in the Summary.
summary = allorders[[
    'Name', 
    'Number of people in household', 
    'Pickup Time',
    'Address', 
    'Email', 
    'Phone', 
    'MEATS/FROZEN ITEMS', 
    'REFRIGERATED ITEMS', 
    'FRESH PRODUCE'
]]
summary = summary.sort_values(by = 'Pickup Time') 

In [20]:
# Build the report using Jinja2.
output = coversheet.render({'received': received, 'printed': printed, 'date': title_date})
output += formatshoppers(title_date, allorders) 
output += sectionheader
output += formatshoppers(title_date, summary)

html = orders.render({'title': name, 'body': output})

In [15]:
# Write out the file to print.
with open(upload_file, 'w') as f:
    f.write(html)
print('Wrote {file}'.format(file = upload_file))

Wrote C:\Users\ralph\Downloads\NCC Food Pantry Orders.html


In [16]:
# Optional: Display the report here.
import IPython
IPython.display.HTML(html)

Name,James Page
Number of people in household,2
Address,7 penstock wayNewmarket NH 03857
Email,jamespage130@gmail.com
Phone,6035198877
Pickup Time,"Thursday, 4 - 6pm"
"Indicate any dietary restrictions (Peanut allergy, low sodium, vegetarian only, etc.):",
MEATS/FROZEN ITEMS,"Boneless Chicken (Breasts, etc.)Ground TurkeyGround BeefHotdogsSausagePulled Pork"
REFRIGERATED ITEMS,MilkEggsCheeseYogurtCottage CheeseDeli MeatJicama Wraps (taco shell sized)
FRESH PRODUCE,"Salad Greens / LettuceCarrotsBell PeppersTomatoesCucumbersPotatoesOnionsCornSquashSide Dish VeggiesApplesGrapefuitLemons/LimesFresh Herbs (Basil, Thyme, Sage, etc)"
BREADS,Sliced Wheat BreadHamburger BunsHotdog RollsBagelsEnglish Muffins
